In [ ]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition
from gillespy2 import SSACSolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import ODECSolver
# from gillespy2 import TauHybridSolver

# Toggle_Switch

In [ ]:
class ToggleSwitch(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Toggle_Switch")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alpha1", expression="10"))
        self.add_parameter(Parameter(name="alpha2", expression="10"))
        self.add_parameter(Parameter(name="beta", expression="2"))
        self.add_parameter(Parameter(name="gamma", expression="2"))
        self.add_parameter(Parameter(name="mu", expression="1"))

        # Variables
        self.add_species(Species(name="A", initial_value=2, mode="discrete"))
        self.add_species(Species(name="B", initial_value=2, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="cu", reactants={}, products={'A': 1}, propensity_function="alpha1/(1+pow(B, beta))"))
        self.add_reaction(Reaction(name="cv", reactants={}, products={'B': 1}, propensity_function="alpha2/(1+pow(A, gamma))"))
        self.add_reaction(Reaction(name="du", reactants={'A': 1}, products={}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="dv", reactants={'B': 1}, products={}, rate=self.listOfParameters["mu"]))

        # Timespan
        self.timespan(np.arange(0, 250, 1))

In [ ]:
model = ToggleSwitch()

# Simulation Parameters

In [ ]:
def configure_simulation():
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":20,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

# Post Processing

## Feature extraction function

In [ ]:
# What value(s) do you want to extract from the simulation trajectory
def population_at_last_timepoint(c, res):
    if c.verbose:
        print(f'population_at_last_timepoint {c.variable_of_interest}={res[c.variable_of_interest][-1]}')
    return res[c.variable_of_interest][-1]

## Aggregation function

In [ ]:
# How do we combine the values from multiple trajectores
def mean_std_of_ensemble(c, data):
    a = np.average(data)
    s = np.std(data)
    if c.verbose:
        print(f'mean_std_of_ensemble m:{a} s:{s}')
    return (a, s)

# Parameter Sweep

In [ ]:
class ParameterSweep1D():

    def run(c, kwargs, verbose=False):
        c.verbose = verbose
        fn = c.feature_extraction
        ag = c.ensemble_aggragator
        data = np.zeros((len(c.p1_range), 2)) # mean and std
        for i, v1 in enumerate(c.p1_range):
            if c.verbose:
                print(f'running {c.p1}={v1}')
            if(c.number_of_trajectories > 1):
                tmp_results = model.run(**kwargs, variables={c.p1:v1})
                (m, s) = ag([fn(x) for x in tmp_results])
                data[i, 0] = m
                data[i, 1] = s
            else:
                tmp_result = model.run(**kwargs, variables={c.p1:v1})
                data[i, 0] = c.feature_extraction(tmp_result)
        c.data = data


    def plot(c):
        from matplotlib import pyplot as plt
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.title(f'Parameter Sweep - Variable:{c.variable_of_interest}')
        plt.errorbar(c.p1_range, c.data[:, 0], c.data[:, 1])
        plt.xlabel(c.p1, fontsize=16, fontweight='bold')
        plt.ylabel('Population', fontsize=16, fontweight='bold')


    def plotplotly(c, return_plotly_figure=False):
        from plotly.offline import iplot
        import plotly.graph_objs as go

        visible = c.number_of_trajectories > 1
        error_y = dict(type='data', array=c.data[:, 1], visible=visible)

        trace_list = [go.Scatter(x=c.p1_range, y=c.data[:, 0], error_y=error_y)]

        title = dict(text=f'<b>Parameter Sweep - Variable: {c.variable_of_interest}</b>', x=0.5)
        yaxis_label = dict(title='<b>Population</b>')
        xaxis_label = dict(title=f'<b>{c.p1}</b>')

        layout = go.Layout(title=title, xaxis=xaxis_label, yaxis=yaxis_label)

        fig = dict(data=trace_list, layout=layout)

        if return_plotly_figure:
            return fig
        iplot(fig)

In [ ]:
class ParameterSweepConfig(ParameterSweep1D):
    # What class defines the GillesPy2 model
    model = ToggleSwitch()
    # ENTER PARAMETER HERE
    p1 = 'alpha1'
    # ENTER START VALUE FOR P1 RANGE HERE
    p1_min = 0.5 * float(eval(model.get_parameter(p1).expression))
    # ENTER END VALUE FOR P1 RANGE HERE
    p1_max = 1.5 * float(eval(model.get_parameter(p1).expression))
    # ENTER THE NUMBER OF STEPS FOR P1 HERE
    p1_steps = 11
    p1_range = np.linspace(p1_min, p1_max, p1_steps)
    # ENTER VARIABLE OF INTEREST HERE
    variable_of_interest = 'A'
    number_of_trajectories = 20
    # What feature of the simulation are we examining
    feature_extraction = population_at_last_timepoint
    # for ensemble resutls: how do we aggreggate the values
    ensemble_aggragator = mean_std_of_ensemble

In [ ]:
kwargs = configure_simulation()
ps = ParameterSweepConfig()
%time ps.run(kwargs)

# Visualization

In [ ]:
ps.plot()

In [ ]:
ps.plotplotly()